In [5]:
import requests
import json
import asyncio
from aiohttp import ClientSession
from web3 import Web3
from datetime import datetime
from pprint import pprint
import telebot

In [8]:
# Addresses
# Wemix 3.0 network
wemix_addresses = [
    "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
    "0xEaB8e0E07e80c35Fef600C988EC0121b1317696b",
    "0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273",
    "0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76"
]

# Ethereum network
eth_addresses = [
    "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
    "0x62271357211D8A325165d61b60F0EdcACC05Bd3e",
    "0x6225C189f583fC2E17f5Be36A6A9db4d3978872D"
]

eth_contract_addresses = [
    "0x2c69095d81305F1e3c6ed372336D407231624CEa",
    "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
]
    
# Polygon network
0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F

497243617554757957071720515671435324516127292175

In [9]:
# API parameters for WEMIX 3.0 network call
url_wemix = "https://api.wemix.com"

In [ ]:
# Calling wemix addresses
addresses = wemix_addresses

In [10]:
# Main function to call on the token balances from main rpcs. 
def get_balance(address, url, block_number=None):
    
    block_param = "latest" if block_number is None else hex(block_number)
    
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "eth_getBalance",
        "params": [address, block_param]
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)['result']

    result_1 = int(result, 16)
    results_2 = result_1/1e18
    
    return results_2

In [ ]:
wemix_balance = []
for address in wemix_addresses:
    balance = get_balance(address, url_wemix, block_number_wemix)
    wemix_balance.append(balance)

In [ ]:
wemix_balance

In [11]:
# Get block number asyncio version
async def get_block_number(session, url):
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method" : "eth_blockNumber",
        "params" : []
    }
    headers = {"Content-Type": "application/json"}
    async with session.post(url, json=payload, headers=headers) as response:
        result = await response.json()
        return int(result['result'], 16)

In [12]:
# Get block number
def get_block_number(url):
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method" : "eth_blockNumber",
        "params" : []
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)["result"]
    return int(result, 16)

In [ ]:
# Get block number timestamp asyncio version
async def get_block_timestamp(session, block_number, url):
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method" : "eth_getBlockByNumber",
        "params" : [hex(block_number), False]
    }
    headers = {"Content-Type": "application/json"}
    async with session.post(url, json=payload, headers=headers) as response:
        result = await response.json()
        return int(result["result"]["timestamp"], 16)

In [13]:
# Get block number timestamp
def get_block_timestamp(block_number, url):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "eth_getBlockByNumber",
        "params": [hex(block_number), False]
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)["result"]
    return int(result["timestamp"], 16)

In [ ]:
# Ethereum rpc node call for specific token balance asyncio version
async def get_token_balance(session, wallet, token_contract, url):
    function_signature = "0x70a08231"
    padded_wallet = "0" * (64 - len(wallet[2:])) + wallet[2:]
    data = function_signature + padded_wallet
    
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method": "eth_call",
        "params": [
                {
                "to" : token_contract,
                "data": data
            },
            "latest"
        ]
    }
    
    headers = {"Content-Type": "application/json"}
    
    async with session.post(url, json=payload, headers=headers) as response:
        result = await response.json()
        result_1 = int(result['result'], 16)
        if token_contract == "0x2c69095d81305F1e3c6ed372336D407231624CEa":
            result_2 = result_1/1e18
        if token_contract == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48":
            result_2 = result_1/1e6
        
        return result_2

In [14]:
# Ethereum rpc node call for specific token balance
def get_token_balance(wallet, token_contract, url, block_number=None):
    function_signature = "0x70a08231"
    padded_wallet = "0" * (64 - len(wallet[2:])) + wallet[2:]
    data = function_signature + padded_wallet
    
    block_param = "latest" if block_number is None else hex(block_number)
    
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method": "eth_call",
        "params": [
                {
                "to" : token_contract,
                "data": data
            },
            block_param
        ]
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)['result']
    result_1 = int(result, 16)
    if token_contract == "0x2c69095d81305F1e3c6ed372336D407231624CEa":
        result_2 = result_1/1e18
    if token_contract == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48":
        result_2 = result_1/1e6
        
    return result_2

In [ ]:
# Main function to call the token addresses
async def main():
    # Wemix addresses
    wemix_addresses = [
        "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
        "0xEaB8e0E07e80c35Fef600C988EC0121b1317696b",
        "0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273",
        "0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76"
    ]

    # Ethereum network
    eth_addresses = [
        "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
        "0x62271357211D8A325165d61b60F0EdcACC05Bd3e",
        "0x6225C189f583fC2E17f5Be36A6A9db4d3978872D"
    ]

    # Ethereum token contracts
    eth_contract_addresses = [
        "0x2c69095d81305F1e3c6ed372336D407231624CEa",
        "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
    ]

    # Ethereum node url
    url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"
    
    async with ClientSession() as session:
        tasks = []
        for address in eth_addresses:
            for token_contract in eth_contract_addresses:
                task = asyncio.ensure_future(get_token_balance(session, address, token_contract, url_eth))
                tasks.append(task)
                
        balances = await asyncio.gather(*tasks)
        
        for i, address in enumerate(eth_addresses):
            for j, token_contract in enumerate(eth_contract_addresses):
                balance = balances[i * len(eth_contract_addresses) + j]
                print(f"token balance of {address} for contract {token_contract}: {balance}")


In [ ]:
url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"
result_eth = []
for address in eth_addresses:
    for token_contract in eth_contract_addresses:
        token_balance = get_token_balance(address, token_contract, url_eth)
        result_eth.append(token_balance)

In [ ]:
result_eth

In [ ]:
# Get the block numbers and timestamps
block_number_wemix = get_block_number(url_wemix)
block_number_eth = get_block_number(url_eth)
block_timestamp = get_block_timestamp(block_number_eth, url_eth)
block_datetime = datetime.fromtimestamp(block_timestamp)

print(f"Block number (WEMIX): {block_number_wemix}")
print(f"Block number (ETH): {block_number_eth}")
print(f"Block timestamp: {block_timestamp} ({block_datetime})")

In [15]:
# Main calling program

# URL for WEMIX 3.0 & ETH network call
url_wemix = "https://api.wemix.com"
url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"

# Get WEMIX balance and block number for wemix chain first
count = 0
wemix_balance = {}
previous_balance = []
block_number_wemix = get_block_number(url_wemix)
message_lines = [f"WEMIX Network"]
message_lines.append(f"Block number [WEMIX] :: {block_number_wemix}")
for address in wemix_addresses:
    balance = get_balance(address, url_wemix, block_number_wemix)
    #wemix_balance.append(balance)
    wemix_balance[address] = balance
    message_lines.append(f"{address} :: {balance} WEMIX ")

# Get balance on ETH wallet address
block_number_eth = get_block_number(url_eth)
message_lines.append(f"Ethereum Network")
message_lines.append(f"Block number [Ethereum] :: {block_number_eth}")
block_timestamp = get_block_timestamp(block_number_eth, url_eth)
block_datetime = datetime.fromtimestamp(block_timestamp)
result_eth = {}
for address in eth_addresses:
    for token_contract in eth_contract_addresses:
        token_balance = get_token_balance(address, token_contract, url_eth, block_number_eth)
        #result_eth.append(token_balance)
        result_eth[address] = token_balance
        if token_contract == '0x2c69095d81305F1e3c6ed372336D407231624CEa' :
            message_lines.append(f"{address} :: {token_balance} WEMIX")
        if token_contract == '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48' and token_balance != 0:
            message_lines.append(f"{address} :: {token_balance} USDC")

        
print(f"Block number (WEMIX): {block_number_wemix}")
print(wemix_balance)
print(f"Block number (ETH): {block_number_eth}")
print(result_eth)
print(f"Block timestamp: {block_timestamp} ({block_datetime})")

Block number (WEMIX): 15403983
{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 4549639.4008327, '0xEaB8e0E07e80c35Fef600C988EC0121b1317696b': 99999.99999999999, '0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273': 0.987599399999876, '0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76': 208.958334}
Block number (ETH): 17058683
{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 55610.0, '0x62271357211D8A325165d61b60F0EdcACC05Bd3e': 0.0, '0x6225C189f583fC2E17f5Be36A6A9db4d3978872D': 0.0}
Block timestamp: 1681638671 (2023-04-16 18:51:11)


In [16]:
wemix_balance

{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 4549639.4008327,
 '0xEaB8e0E07e80c35Fef600C988EC0121b1317696b': 99999.99999999999,
 '0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273': 0.987599399999876,
 '0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76': 208.958334}

In [17]:
result_eth

{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 55610.0,
 '0x62271357211D8A325165d61b60F0EdcACC05Bd3e': 0.0,
 '0x6225C189f583fC2E17f5Be36A6A9db4d3978872D': 0.0}

In [19]:
# Main calling program

# URL for WEMIX 3.0 & ETH network call
url_wemix = "https://api.wemix.com"
url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"

# Get WEMIX balance and block number for wemix chain first
count = 0
previous_balance_wemix = {}
block_number_wemix = get_block_number(url_wemix)
message_lines = [f"WEMIX Network"]
message_lines.append(f"Block number [WEMIX] :: {block_number_wemix}")
for address in wemix_addresses:
    balance = get_balance(address, url_wemix, block_number_wemix)
    #wemix_balance.append(balance)
    previous_balance_wemix[address] = balance
    message_lines.append(f"{address} :: {balance} WEMIX ")

# Get balance on ETH wallet address
block_number_eth = get_block_number(url_eth)
message_lines.append(f"Ethereum Network")
message_lines.append(f"Block number [Ethereum] :: {block_number_eth}")
block_timestamp = get_block_timestamp(block_number_eth, url_eth)
block_datetime = datetime.fromtimestamp(block_timestamp)
previous_balance_eth = {}
for address in eth_addresses:
    for token_contract in eth_contract_addresses:
        token_balance = get_token_balance(address, token_contract, url_eth, block_number_eth)
        #result_eth.append(token_balance)
        previous_balance_eth[address] = token_balance
        if token_contract == '0x2c69095d81305F1e3c6ed372336D407231624CEa' :
            message_lines.append(f"{address} :: {token_balance} WEMIX")
        if token_contract == '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48' and token_balance != 0:
            message_lines.append(f"{address} :: {token_balance} USDC")

        
print(f"Block number (WEMIX): {block_number_wemix}")
print(previous_balance_wemix)
print(f"Block number (ETH): {block_number_eth}")
print(previous_balance_eth)
print(f"Block timestamp: {block_timestamp} ({block_datetime})")

Block number (WEMIX): 15404180
{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 4549639.4008327, '0xEaB8e0E07e80c35Fef600C988EC0121b1317696b': 99999.99999999999, '0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273': 0.987599399999876, '0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76': 208.958334}
Block number (ETH): 17058699
{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 55610.0, '0x62271357211D8A325165d61b60F0EdcACC05Bd3e': 0.0, '0x6225C189f583fC2E17f5Be36A6A9db4d3978872D': 0.0}
Block timestamp: 1681638875 (2023-04-16 18:54:35)


In [20]:
previous_balance_wemix

{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 4549639.4008327,
 '0xEaB8e0E07e80c35Fef600C988EC0121b1317696b': 99999.99999999999,
 '0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273': 0.987599399999876,
 '0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76': 208.958334}

In [21]:
previous_balance_eth

{'0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F': 55610.0,
 '0x62271357211D8A325165d61b60F0EdcACC05Bd3e': 0.0,
 '0x6225C189f583fC2E17f5Be36A6A9db4d3978872D': 0.0}

In [25]:
# Compare the changes of the previous to new

for address in wemix_addresses:
    print(f'Wemix balance: {wemix_balance[address]}')
    print(f'Previous balance: {previous_balance_wemix[address]}')
    if wemix_balance[address] != previous_balance_wemix[address]:
        print(f'New transaction found on Block number: {block_number_wemix}!')
        transactions = get_transactions(address, url_wemix, block_number_wemix)
        print(transactions)
        


Wemix balance: 4549639.4008327
Previous balance: 4549639.4008327
Wemix balance: 99999.99999999999
Previous balance: 99999.99999999999
Wemix balance: 0.987599399999876
Previous balance: 0.987599399999876
Wemix balance: 208.958334
Previous balance: 208.958334


In [50]:
# Get transaction on WEMIX network
def get_transactions_wemix(address, url, block_number):
    transactions = []
    for block_index in range(block_number - 12, block_number+1):
    
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getBlockByNumber",
            "params": [hex(block_index), True],
            "id": 1
        }
        response = requests.post(url, json=payload)
        block = response.json()['result']
        
        for tx in block["transactions"]:
            if tx["from"].lower() == address.lower() or tx["to"].lower() == address.lower():
                transactions.append(tx)
    
    return transactions
            

In [53]:
# Get transaction from Ethereum network
def get_transactions_eth(address, url, block_number):
    transactions = []
    for block_index in range(block_number - 12, block_number+1):
    
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getBlockByNumber",
            "params": [hex(block_index), True],
            "id": 1
        }
        response = requests.post(url, json=payload)
        block = response.json()['result']
        
        for tx in block["transactions"]:
            if tx["from"] == address or tx["to"] == address:
                transactions.append(tx)
    
    return transactions

In [54]:
address = '0x57192cca8b8e4beb77f3466c6d0550e64cc53b0f'
transaction = get_transactions_eth(address,url_eth, 17014626)
transaction

[{'accessList': [],
  'blockHash': '0x262f75ec1834184e6475db2017f8e55c26397a505cdb4335534f9f3ed599298d',
  'blockNumber': '0x1039f62',
  'chainId': '0x1',
  'from': '0x57192cca8b8e4beb77f3466c6d0550e64cc53b0f',
  'gas': '0xae13',
  'gasPrice': '0x4d210cb46',
  'hash': '0xcedd78e657c9b3ad8402fb92ba617d75b98d5916da5b95d7d334527527e4fd4a',
  'input': '0x241dc2df0000000000000000000000002c69095d81305f1e3c6ed372336d407231624cea000000000000000000000000c5ffda0067c63bc94d6fa1248c2fc32eb3858a760000000000000000000000000000000000000000000000056bc75e2d631000000000000000000000000000000000000000000000000000000000000000000457',
  'maxFeePerGas': '0xd88994930',
  'maxPriorityFeePerGas': '0x3f9920c0',
  'nonce': '0x5',
  'r': '0xb94d9b4325f4f171ddc1dc637eb82563aac34b8e713b2e86a8d1c4982d853217',
  's': '0x3ad344b5e930b81e9a8fdc453d4ab05e644e0cb53b5bb4ae6fe4e7b35fdff191',
  'to': '0x765277eebeca2e31912c9946eae1021199b39c61',
  'transactionIndex': '0x43',
  'type': '0x2',
  'v': '0x0',
  'value': '0x0'}]

In [48]:
block_number = transaction[0]["from"]
block_number

'0x44305dbe5566ae452788dd22081699ee9de13a72'

In [ ]:
# Telegram bot send function
def send_telegram_message(bot_token, chat_id, message):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message
    }
    response = requests.post(url, json=payload)
    return response.json()

In [ ]:
message_lines

In [ ]:
# Telegram bot 
#5961430353:AAENiehDUvjJ49K9Kw-fUAiGBhbV3jtPqrg

#Initialize bot:
bot_token = "5961430353:AAENiehDUvjJ49K9Kw-fUAiGBhbV3jtPqrg"
chat_id = '1628044387'

message = "\n".join(message_lines)
send_telegram_message(bot_token, chat_id, message)

In [ ]:
# Api for my bot
https://api.telegram.org/bot5961430353:AAENiehDUvjJ49K9Kw-fUAiGBhbV3jtPqrg/getMe
        
        

In [56]:
bot_token = "5961430353:AAENiehDUvjJ49K9Kw-fUAiGBhbV3jtPqrg"

def get_updates(bot_token):
    url = f"https://api.telegram.org/bot{bot_token}/getUpdates"
    response = requests.get(url)
    return response.json()

updates = get_updates(bot_token)
pprint(updates)

{'ok': True,
 'result': [{'message': {'chat': {'first_name': 'WH',
                                  'id': 1628044387,
                                  'type': 'private',
                                  'username': 'wonheejo'},
                         'date': 1681633918,
                         'from': {'first_name': 'WH',
                                  'id': 1628044387,
                                  'is_bot': False,
                                  'language_code': 'ko',
                                  'username': 'wonheejo'},
                         'message_id': 38,
                         'text': 'asdf'},
             'update_id': 107535527},
            {'message': {'chat': {'first_name': 'WH',
                                  'id': 1628044387,
                                  'type': 'private',
                                  'username': 'wonheejo'},
                         'date': 1681633925,
                         'from': {'first_name': 'WH',
           